## Process that downloads the historical data of stock market of Technology industry.

#### Imports

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from pandas_datareader import data as pdr
import json
import os
import psycopg2
import requests
import time
import fix_yahoo_finance as yf

#### Global variables and constants

In [2]:
PATH_PROJECT = os.getcwd()
POSTGRES_DB = 'stock-tracker'
POSTGRES_SCHM = 'public'
POSTGRES_USER = 'postgres'
POSTGRES_PASS = 'Yj18pQ-$20'

queued_stocks = []

#### 1. Download the symbol list of Technology industry

In [3]:
def technology_stocks():
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'
    }
    try:
        base_url = 'https://api.iextrading.com/1.0/stock/market/collection/sector?collectionName=Technology'
        r = requests.get(base_url, headers = headers)
        if r.status_code == 200:
            src = BeautifulSoup(r.text, 'lxml')
            stock_list = json.loads(src.find('p').text)
            stocks = [stock['symbol'] for stock in stock_list]
            return stocks
        else:
            return None
    except Exception as ex:
        return None

#### 2. Download the historical data of the stock symbols of Technology industry (one by one to ease the upload to DB)

In [4]:
def historical_report(stock, start_date, end_date, code):
    yf.pdr_override()
    try:
        data = pdr.get_data_yahoo(
            stock, 
            start = start_date, 
            end = end_date, 
            progress = False
        )
        data['Stock'] = stock
        return data
    except Exception as ex:
        print('Yahoo Finance collapse downloading %s data. Adding to queue to continue once the correct downloads end...' % stock)
        if code == 0:
            queued_stocks.append(stock)

#### 3. Upload historical data to PostgreSQL database

In [5]:
def upload_historical(name, filename):
    conn = None
    commands = ("""SELECT * FROM information_schema.tables
            WHERE table_name = '%s'""" % name, """CREATE TABLE %s (
            open        double precision NOT NULL,
            high        double precision NOT NULL,
            low         double precision NOT NULL,
            close       double precision NOT NULL,
            adj_close   double precision NOT NULL,
            volume      integer NOT NULL,
            stock       varchar(10) NOT NULL,
            date_d      varchar(10) NOT NULL
        )""" % (name))
    try:
        conn = psycopg2.connect('dbname=%s user=%s password=%s' % (POSTGRES_DB, POSTGRES_USER, POSTGRES_PASS))
        cur = conn.cursor()
        cur.execute(commands[0])
        if (bool(cur.rowcount)):
            print('Table already exists.')
        else:
            cur.execute(commands[1])
        with open(PATH_PROJECT + '/data/%s.csv' % filename) as f:
            next(f)
            cur.copy_from(f, name, sep = '\t')
        cur.close()
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as ex:
        print(str(ex))
    finally:
        if conn is not None:
            conn.close()

#### Main execution of the process

In [6]:
stock_symbols = technology_stocks()
if stock_symbols != None:
    start_date = '1990-01-01'
    end_date = '2017-12-31'
    historical_data_1 = [historical_report(stock, start_date, end_date, 0) for stock in stock_symbols]
    historical_data_2 = [historical_report(stock, start_date, end_date, 1) for stock in queued_stocks]
    historical_data_1_df = pd.concat(historical_data_1)
    historical_data_2_df = pd.concat(historical_data_2)
    historical_data = pd.concat([historical_data_1_df, historical_data_2_df])
    historical_data['date_d'] = historical_data.index
    historical_data.columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume', 'stock', 'date_d']
    historical_data.to_csv(PATH_PROJECT + '/data/stocks_historical.csv', index = False, sep = '\t', encoding = 'utf-8')
    upload_historical('stocks_historical', 'stocks_historical')

Yahoo Finance collapse downloading CBLK data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading DBX data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading GSKY data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading LEXEB data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading EVH data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading IIIV data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading LKM data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading BDC-B data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading PRSP data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downlo

Yahoo Finance collapse downloading DOCU data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading EVOP data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading SONO data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading UXIN data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading ZS data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading EVER data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading KODK+ data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading RDVT data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading HQCL data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downlo

In [7]:
historical_data

,open,high,low,close,adj_close,volume,stock,date_d
Date,,,,,,,,
2014-04-04,40.000000,40.799999,34.000000,34.000000,34.000000,10970100,GRUB,2014-04-04
2014-04-07,33.980000,35.869999,33.060001,34.799999,34.799999,1152200,GRUB,2014-04-07
2014-04-08,35.250000,36.619999,34.599998,35.990002,35.990002,510700,GRUB,2014-04-08
2014-04-09,36.099998,36.500000,34.610001,35.310001,35.310001,455300,GRUB,2014-04-09
2014-04-10,35.369999,35.369999,30.815001,30.910000,30.910000,754100,GRUB,2014-04-10
2014-04-11,30.780001,34.470001,29.860001,33.639999,33.639999,1185600,GRUB,2014-04-11
2014-04-14,34.080002,36.560001,32.880001,35.160000,35.160000,856200,GRUB,2014-04-14
2014-04-15,35.580002,35.580002,31.340000,35.000000,35.000000,729600,GRUB,2014-04-15
2014-04-16,33.900002,37.700001,33.619999,37.520000,37.520000,869600,GRUB,2014-04-16


#### 4. Check if there are stocks without information

In [8]:
def check_historical(stock, name, cur):
    cur.execute("SELECT * FROM %s WHERE stock = '%s'" % (name, stock))
    if (len(cur.fetchall()) == 0):
        return stock

In [9]:
stock_symbols = technology_stocks()
if stock_symbols != None:
    conn = None
    try:
        conn = psycopg2.connect('dbname=%s user=%s password=%s' % (POSTGRES_DB, POSTGRES_USER, POSTGRES_PASS))
        cur = conn.cursor()
        queued_stocks = [check_historical(stock, 'stocks_historical', cur) for stock in stock_symbols]
        cur.close()
    except (Exception, psycopg2.DatabaseError) as ex:
        print(str(ex))
    finally:
        if conn is not None:
            conn.close()
    start_date = '1990-01-01'
    end_date = '2017-12-31'
    queued_stocks = list(filter(None, queued_stocks))
    remaining_historical_df = pd.concat([historical_report(stock, start_date, end_date, 1) for stock in queued_stocks])
    remaining_historical_df['date_d'] = remaining_historical_df.index
    remaining_historical_df.columns = ['open', 'high', 'low', 'close', 'adj_close', 'volume', 'stock', 'date_d']
    remaining_historical_df.to_csv(PATH_PROJECT + '/data/stocks_historical_r.csv', index = False, sep = '\t', encoding = 'utf-8')
    upload_historical('stocks_historical', 'stocks_historical_r')

[None, None, None, None, 'CBLK', None, None, None, None, None, None, 'DBX', None, 'GSKY', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'LEXEB', None, None, None, None, None, None, None, 'EVH', None, 'IIIV', 'LKM', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'BDC-B', None, 'PRSP', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'BV', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'VPG', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'CLRO', None, N

Yahoo Finance collapse downloading MEET data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading LGL data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading CANG data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading WISA data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading PVTL data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading AVLR data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading TAOP data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading BILI data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloading GSB data. Adding to queue to continue once the correct downloads end...
Yahoo Finance collapse downloa

In [10]:
remaining_historical_df

,open,high,low,close,adj_close,volume,stock,date_d
Date,,,,,,,,
1995-03-06,10.500000,10.500000,7.000000,8.000000,8.000000,1100,USATP,1995-03-06
1995-03-07,8.000000,8.000000,8.000000,8.000000,8.000000,0,USATP,1995-03-07
1995-03-08,8.000000,8.000000,8.000000,8.000000,8.000000,0,USATP,1995-03-08
1995-03-09,8.000000,8.000000,8.000000,8.000000,8.000000,0,USATP,1995-03-09
1995-03-10,8.000000,8.000000,8.000000,8.000000,8.000000,0,USATP,1995-03-10
1995-03-13,8.000000,8.000000,8.000000,8.000000,8.000000,0,USATP,1995-03-13
1995-03-14,8.000000,8.000000,7.875000,7.875000,7.875000,5100,USATP,1995-03-14
1995-03-15,7.875000,7.875000,7.875000,7.875000,7.875000,0,USATP,1995-03-15
1995-03-16,7.875000,7.875000,7.875000,7.875000,7.875000,0,USATP,1995-03-16
